In [1]:
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostClassifier

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split

In [3]:
df1 = pd.read_excel('data-MDPI-Random-Forest.xlsx', sheet_name=0)
df2 = pd.read_excel('data-MDPI-Random-Forest.xlsx', sheet_name=1) 

In [4]:
print(df1.columns)
print(df2.columns)

Index(['Dataset Type', 'Extrusion Temperature (in C)', 'Layer Height (mm)',
       'Shell Thickness (mm)', 'Expt', 'LR', 'RFR', 'ABR', 'GBR', 'XGBR'],
      dtype='object')
Index(['Dataset Type', 'Layer Thickness (mm)', 'XY', 'XZ', 'YZ', 'Expt', 'LR',
       'RFR', 'ABR', 'GBR', 'XGBR'],
      dtype='object')


In [17]:
max(df1['Shell Thickness (mm)'])

1.2

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df1.loc[:,['Extrusion Temperature (in C)', 'Layer Height (mm)', 'Shell Thickness (mm)']], \
df1.loc[:, 'Expt'], train_size = 19, shuffle = False)


In [6]:
model = xgb.XGBRegressor(n_estimators=300, booster='gbtree', learning_rate=0.3, max_depth=6)
#model = GradientBoostingRegressor(n_estimators=1200,learning_rate=1.0)
#model = AdaBoostClassifier(n_estimators=100,learning_rate=1.0)

In [7]:
model.fit(X_train.to_numpy(),y_train.to_numpy())

predictions = model.predict(X_test.to_numpy())

In [8]:
result = df1.loc[df1['Dataset Type'] == 'Test', ['Dataset Type', 'Expt', 'XGBR']].reset_index(drop=True)
result['Predictions'] = pd.Series(predictions)
result

,Dataset Type,Expt,XGBR,Predictions
0,Test,43.254,46.7304,46.635403
1,Test,57.332,50.6326,51.612251
2,Test,53.981,32.7860,28.132568
3,Test,52.274,55.4502,52.998081
4,Test,26.588,37.1562,38.928310
5,Test,47.586,52.4950,51.702663
6,Test,48.307,53.0042,52.522217
7,Test,52.299,55.6377,54.570969
8,Test,51.032,45.9100,49.071541


In [9]:
print("r2 score (author) : ", r2_score(result['Expt'].to_numpy(), result['XGBR'].to_numpy()))
print("r2 score (amt) : ", r2_score(result['Expt'].to_numpy(), result['Predictions'].to_numpy()))
print(mean_squared_error(result['Expt'].to_numpy(), result['XGBR'].to_numpy()))
#print(pow(mean_squared_error(y_test.to_numpy(), predictions), 0.5))

r2 score (author) :  -0.09460927175004796
r2 score (amt) :  -0.3981467194216155
79.05713450333333


In [10]:
print("r2 score (author) : ", r2_score(df1['Expt'].to_numpy(), df1['XGBR'].to_numpy()))
print(mean_squared_error(df1['Expt'].to_numpy(), df1['XGBR'].to_numpy()))
print("r2 score (amt) : ", r2_score(df1['Expt'].to_numpy(), df1['Predictions'].to_numpy()))
#print(pow(mean_squared_error(y_test.to_numpy(), predictions), 0.5))

r2 score (author) :  0.6045015612282447
25.41122491107143


KeyError: 'Predictions'

In [ ]:
# Rewriting r2score definiton
def getRSquared(y, predictions):
    ssr = 0
    for i in range(len(y)):
        ssr = ssr + pow(y[i] - predictions[i], 2)
    sst = 0
    ymean = 0
    for y_i in y:
        ymean = ymean + y_i
    ymean = ymean / len(y)
    for i in range(len(y)):
        sst = sst + pow(y[i] - ymean, 2)
    return 1 - ssr / sst